## Importing Necessary Libraries

In [73]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
import pickle
import pandas as pd

## Data Preprocessing

In [74]:
df = pd.read_csv('/content/drive/MyDrive/data/Online Retail.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [80]:
data = df[['InvoiceNo', 'Description', 'Quantity', 'UnitPrice', 'Country', 'InvoiceDate']]
data.head()

,InvoiceNo,Description,Quantity,UnitPrice,Country,InvoiceDate
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,United Kingdom,12/1/2010 8:26
1,536365,WHITE METAL LANTERN,6,3.39,United Kingdom,12/1/2010 8:26
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2.75,United Kingdom,12/1/2010 8:26
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,United Kingdom,12/1/2010 8:26
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,United Kingdom,12/1/2010 8:26


In [81]:
data.shape

(541909, 6)

In [82]:
data['Country'].value_counts()

United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

### Handling Categorical Data

In [83]:
encoder = LabelEncoder()
data['CountryNumeric'] = encoder.fit_transform(data['Country'])
data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,InvoiceNo,Description,Quantity,UnitPrice,Country,InvoiceDate,CountryNumeric
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,United Kingdom,12/1/2010 8:26,36
1,536365,WHITE METAL LANTERN,6,3.39,United Kingdom,12/1/2010 8:26,36
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2.75,United Kingdom,12/1/2010 8:26,36
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,United Kingdom,12/1/2010 8:26,36
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,United Kingdom,12/1/2010 8:26,36


In [84]:
data.to_csv('preprocessed_data_for_nn_model.csv')

## Modelling

In [35]:
final_df_for_nn_model = data[['Quantity', 'UnitPrice', 'Country']]
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)
model.fit(final_df_for_nn_model.values)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                 radius=1.0)

## Testing Model

In [36]:
test = final_df_for_nn_model.iloc[0].values.reshape(1, -1)
test

array([[ 6.  ,  2.55, 36.  ]])

In [37]:
ind = model.kneighbors(test, return_distance=False)
ind

array([[489240, 456182, 177199, 422195, 489239, 286795, 489283,  83563,
         17019, 177640, 177696, 421805, 421803, 421802, 177839, 177840,
        421702, 539944, 177200,  83775]])

In [38]:
recommended_product = df.iloc[ind[0]]
recommended_product

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
489240,577985,21535,RED RETROSPOT SMALL MILK JUG,6,11/22/2011 12:25,2.55,15116.0,United Kingdom
456182,575680,23311,VINTAGE CHRISTMAS STOCKING,6,11/10/2011 15:23,2.55,13148.0,United Kingdom
177199,552046,22303,COFFEE MUG APPLES DESIGN,6,5/6/2011 9:44,2.55,15235.0,United Kingdom
422195,573069,22301,COFFEE MUG CAT + BIRD DESIGN,6,10/27/2011 13:58,2.55,18077.0,United Kingdom
489239,577985,21531,RED RETROSPOT SUGAR JAM BOWL,6,11/22/2011 12:25,2.55,15116.0,United Kingdom
286795,562043,22970,LONDON BUS COFFEE MUG,6,8/2/2011 10:19,2.55,15342.0,United Kingdom
489283,578006,22300,COFFEE MUG DOG + BALL DESIGN,6,11/22/2011 12:40,2.55,13363.0,United Kingdom
83563,543325,21175,GIN + TONIC DIET METAL SIGN,6,2/7/2011 12:22,2.55,17692.0,United Kingdom
17019,537671,18097C,WHITE TALL PORCELAIN T-LIGHT HOLDER,6,12/8/2010 9:20,2.55,13267.0,United Kingdom
177640,552166,21039,RED RETROSPOT SHOPPING BAG,6,5/6/2011 12:21,2.55,13475.0,United Kingdom


## Making Recommendations

In [62]:
def recommend_product(*product_data):
    ind = model.kneighbors(product_data, return_distance=False)
    return df.iloc[ind[0]][['InvoiceNo', 'Description', 'Quantity', 'UnitPrice', 'Country']]

In [64]:
test = data.iloc[0][['Quantity', 'UnitPrice', 'Country']].values.reshape(1, -1)
input = test.tolist()[0]
recommend_product(input)

,InvoiceNo,Description,Quantity,UnitPrice,Country
489240,577985,RED RETROSPOT SMALL MILK JUG,6,2.55,United Kingdom
456182,575680,VINTAGE CHRISTMAS STOCKING,6,2.55,United Kingdom
177199,552046,COFFEE MUG APPLES DESIGN,6,2.55,United Kingdom
422195,573069,COFFEE MUG CAT + BIRD DESIGN,6,2.55,United Kingdom
489239,577985,RED RETROSPOT SUGAR JAM BOWL,6,2.55,United Kingdom
286795,562043,LONDON BUS COFFEE MUG,6,2.55,United Kingdom
489283,578006,COFFEE MUG DOG + BALL DESIGN,6,2.55,United Kingdom
83563,543325,GIN + TONIC DIET METAL SIGN,6,2.55,United Kingdom
17019,537671,WHITE TALL PORCELAIN T-LIGHT HOLDER,6,2.55,United Kingdom
177640,552166,RED RETROSPOT SHOPPING BAG,6,2.55,United Kingdom


## Model Export and Deployment

In [68]:
file = open('model.pkl', 'wb')
pickle.dump(model, file)

In [69]:
model = pickle.load(open('model.pkl', 'rb'))
model

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                 radius=1.0)

In [87]:
le_name_mapping = dict(zip(encoder.transform(encoder.classes_), encoder.classes_))

In [88]:
le_name_mapping

{0: 'Australia',
 1: 'Austria',
 2: 'Bahrain',
 3: 'Belgium',
 4: 'Brazil',
 5: 'Canada',
 6: 'Channel Islands',
 7: 'Cyprus',
 8: 'Czech Republic',
 9: 'Denmark',
 10: 'EIRE',
 11: 'European Community',
 12: 'Finland',
 13: 'France',
 14: 'Germany',
 15: 'Greece',
 16: 'Hong Kong',
 17: 'Iceland',
 18: 'Israel',
 19: 'Italy',
 20: 'Japan',
 21: 'Lebanon',
 22: 'Lithuania',
 23: 'Malta',
 24: 'Netherlands',
 25: 'Norway',
 26: 'Poland',
 27: 'Portugal',
 28: 'RSA',
 29: 'Saudi Arabia',
 30: 'Singapore',
 31: 'Spain',
 32: 'Sweden',
 33: 'Switzerland',
 34: 'USA',
 35: 'United Arab Emirates',
 36: 'United Kingdom',
 37: 'Unspecified'}